In [69]:
# Greedy
# Runtime: 36 ms, faster than 79.46%
# Refer to https://leetcode.com/problems/text-justification/discuss/24902/Java-easy-to-understand-broken-into-several-functions
from typing import List

class Solution:
    def fullJustify(self, words: List[str], maxWidth: int) -> List[str]:
        # return (rightmost word index, sum of word characters in this line)
        def fillInLine(left: int) -> (int, int):
            nonlocal maxWidth, N
            right = left
            sumOfChars = 0
            # must add the first word's length to avoid over count one word
            while right < N and sumOfChars+len(words[right]) <= maxWidth-(right-left):
                sumOfChars += len(words[right])
                right += 1
            # right - 1 to refund the over counted right index in while loop
            return right-1, sumOfChars
            
        # Three edge cases:
        # 1. only one word in line
        # 2. last line, needs left justify
        # 3. if there are remaining spaces after evenly filled (i.e. totalSpaceNum % gapNum > 0)
        def justify(left: int, right: int, sumOfChars: int) -> str:
            nonlocal maxWidth, N
            gapNum = right-left
            totalSpaceNum = maxWidth-sumOfChars
            if gapNum == 0: # 1. only one word in this line
                return words[left] + ' ' * totalSpaceNum # pad spaces for end of this line
            avgSpaceNum = totalSpaceNum // gapNum
            additionalSpaceNum = totalSpaceNum % gapNum
            cur = left
            res = ''
            while cur < right:
                spaceNum = 1 # 2. last line is left justify
                if right < N-1:
                    # 3. remaining spaces left after evenly filled
                    # we evently filled the first additionalSpaceNum gaps until used up
                    spaceNum = avgSpaceNum + (additionalSpaceNum > 0)
                res += words[cur] + ' ' * spaceNum
                if additionalSpaceNum > 0:
                    additionalSpaceNum -= 1
                cur += 1
            res += words[cur] # add the last word without add space
            if right == N-1: # pad spaces for end of last line
                res += ' ' * (totalSpaceNum-gapNum)
            return res
        
        left = 0
        res = []
        N = len(words)
        while left < N:
            right, sumOfChars = fillInLine(left)
#             print(words[left:right+1], sumOfChars)
            line = justify(left, right, sumOfChars)
            res.append(line)
            left = right + 1
        return res

In [92]:
# Round Robin for space fill
# Runtime: 32 ms, faster than 94.64%

# Once you determine that there are only k words that can fit on a given line, 
# you know what the total length of those words is num_of_letters. Then the rest are 
# spaces, and there are (maxWidth - num_of_letters) of spaces.
# The "or 1" part is for dealing with the edge case len(wordsInLine) == 1.
# https://leetcode.com/problems/text-justification/discuss/24891/Concise-python-solution-10-lines.
from typing import List

class Solution:
    def fullJustify(self, words: List[str], maxWidth: int) -> List[str]:
        lines, wordsInLine, sumOfWords = [], [], 0
        for w in words:
            if sumOfWords + len(wordsInLine) + len(w) > maxWidth:
                # round robin: allocate remaining spaces word by word
                for i in range(maxWidth-sumOfWords):
                    # if word number in this line is 1, just % 1
                    wordsInLine[i % (len(wordsInLine)-1 or 1)] += ' '
                lines.append(''.join(wordsInLine))
                # reset variables for next line
                wordsInLine, sumOfWords = [], 0
            # aggregate this word
            wordsInLine += w,
            sumOfWords += len(w)
        # ljust: right padding space till maxWidth (left justify)
#         return lines + [' '.join(wordsInLine).ljust(maxWidth)]
        lastLine = ' '.join(wordsInLine) + ' '*(maxWidth-sumOfWords-len(wordsInLine)+1)
        lines.append(lastLine)
        return lines

In [93]:
Solution().fullJustify(words=["This", "is", "an", "example", "of", "text", "justification."], maxWidth=16)

['This    is    an', 'example  of text', 'justification.   ']

In [94]:
Solution().fullJustify(words=["What","must","be","acknowledgment","shall","be"], maxWidth=16)

['What   must   be', 'acknowledgment  ', 'shall be         ']

In [95]:
Solution().fullJustify(words=["Science","is","what","we","understand","well","enough","to","explain","to","a","computer.","Art","is","everything","else","we","do"], maxWidth=20)

['Science  is  what we',
 'understand      well',
 'enough to explain to',
 'a  computer.  Art is',
 'everything  else  we',
 'do                   ']